In [ ]:
pip install -r ../requirements.txt --quiet

In [1]:
import json
from tqdm import tqdm
from razdel import sentenize
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
from transformers import MBartTokenizer, MBartForConditionalGeneration
import torch

/Users/wequalwo/LETI/1. NLP/venv/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
device = 'mps' if torch.backends.mps.is_available() else 'cpu'
model_name = "sarahai/ru-sum" #"cointegrated/rut5-base-absum" 

In [4]:
#для "sarahai/ru-sum" 
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name).to(device)
model.eval()

# для "cointegrated/rut5-base-absum"
# model_name = "cointegrated/rut5-base-absum"
# tokenizer = AutoTokenizer.from_pretrained(model_name)
# model = AutoModelForSeq2SeqLM.from_pretrained(model_name).to(device)
# model.eval()

T5ForConditionalGeneration(
  (shared): Embedding(30000, 768)
  (encoder): T5Stack(
    (embed_tokens): Embedding(30000, 768)
    (block): ModuleList(
      (0): T5Block(
        (layer): ModuleList(
          (0): T5LayerSelfAttention(
            (SelfAttention): T5Attention(
              (q): Linear(in_features=768, out_features=768, bias=False)
              (k): Linear(in_features=768, out_features=768, bias=False)
              (v): Linear(in_features=768, out_features=768, bias=False)
              (o): Linear(in_features=768, out_features=768, bias=False)
              (relative_attention_bias): Embedding(32, 12)
            )
            (layer_norm): T5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): T5LayerFF(
            (DenseReluDense): T5DenseGatedActDense(
              (wi_0): Linear(in_features=768, out_features=2048, bias=False)
              (wi_1): Linear(in_features=768, out_features=2048, bias=False)
              (wo):

In [9]:
# def abstractive_summary(text, max_symbols=300):
#     inputs = tokenizer(text, return_tensors="pt", truncation=True, max_length=512).to(device)
#     summary_ids = model.generate(
#           **inputs,
#           max_length=100,
#           min_length=60,
#           num_beams=30,
#           repetition_penalty=3.0,
#           length_penalty=1.2,
#           no_repeat_ngram_size=10,
#           early_stopping=True
#       )
#     summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
#     return summary[:max_symbols]


def improved_abstractive_summary(text, max_symbols=300, device='cuda'):
    inputs = tokenizer(text, return_tensors="pt", truncation=True, max_length=512).to('mps')
    with torch.no_grad():
        summary_ids = model.generate(
            **inputs,
            max_length=150,          # подогнал под размер 300 на глаз
            min_length=60,
            num_beams=30,            # взял прямо жирно, но так мне больше нравится
            temperature=0.9,         # Для чуть большей вариативности
            repetition_penalty=1.5,  # Мягче штрафуем повторения
            length_penalty=1.0,      # Нейтральное влияние на длину
            no_repeat_ngram_size=3,  
            early_stopping=True,
            do_sample=True          
        )
    summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
    return summary[:max_symbols]


In [10]:
with open("ex2.json", "r", encoding="utf-8") as f:
    texts = json.load(f)

summaries = []
for text in tqdm(texts, desc="Summarizing texts"):
    abs_summary = improved_abstractive_summary(text)
    summaries.append(abs_summary)

with open("summaries.json", "w", encoding="utf-8") as f:
    json.dump(summaries, f, ensure_ascii=False, indent=2)

Summarizing texts: 100%|██████████| 5/5 [00:37<00:00,  7.48s/it]


In [15]:
reference_summaries = [
    "Пёс Дик любит смотреть, как я курю. На болоте я нашёл черничную поляну, встал на четвереньки и стал есть ягоды прямо с куста. Дик наблюдал и радовался, а потом сам собрал чернику вокруг избушки.",
    "Скворца у нас любят больше всех. У поэта жил скворец Петруша, он сочинял стихи и печатал их в журналах. За каждую строчку платили рубль и сорок копеек. Петруша подражал голосам и звукам, его трели напоминали весенние песни.",
    "Заяц собрал букет полевых цветов и не знает, кому его подарить. Лисе и волку заячий букет не нужен, у русаков цветы над ушами, у беляков за хвостами. Но этот заяц решил подарить букет, чтобы принести радость.",
    "Ночью в лесу у костра напал страх. Я глядел в огонь, боялся поднять голову. В тишине слышался треск и шёпот, кто-то за спиной шуршал. Рассказ передаёт чувство тревоги и страх перед неизвестным.",
    "Холодный октябрьский ветер пришёл с севера, из тундры с берегов Печоры. Листобой завывал в печной трубе, шевелил осиновую щепу на крыше, бил и трепал деревья, принося дыхание осени и холода."
]

for i in range(len(texts)):
    print(f"Длинна исходного текста: {len(texts[i])}, Длинна реферата: {len(summaries[i])}, Длинна референса: {len(reference_summaries[i])}")

Длинна исходного текста: 1054, Длинна реферата: 163, Длинна референса: 194
Длинна исходного текста: 1101, Длинна реферата: 172, Длинна референса: 223
Длинна исходного текста: 807, Длинна реферата: 181, Длинна референса: 208
Длинна исходного текста: 684, Длинна реферата: 178, Длинна референса: 193
Длинна исходного текста: 740, Длинна реферата: 182, Длинна референса: 190


In [17]:
from rouge import Rouge

rouge = Rouge()

for i, (summary, reference) in enumerate(zip(summaries, reference_summaries)):
    score = rouge.get_scores(summary, reference, avg=False)
    print(f"\nРезультаты для пары {i+1}:")
    print(f"ROUGE-1: {score[0]['rouge-1']}")
    print(f"ROUGE-2: {score[0]['rouge-2']}")
    print(f"ROUGE-L: {score[0]['rouge-l']}")


Результаты для пары 1:
ROUGE-1: {'r': 0.5483870967741935, 'p': 0.6296296296296297, 'f': 0.5862068915755053}
ROUGE-2: {'r': 0.42424242424242425, 'p': 0.5, 'f': 0.45901638847621606}
ROUGE-L: {'r': 0.5483870967741935, 'p': 0.6296296296296297, 'f': 0.5862068915755053}

Результаты для пары 2:
ROUGE-1: {'r': 0.6571428571428571, 'p': 0.8214285714285714, 'f': 0.7301587252204584}
ROUGE-2: {'r': 0.4722222222222222, 'p': 0.5666666666666667, 'f': 0.5151515101928377}
ROUGE-L: {'r': 0.6285714285714286, 'p': 0.7857142857142857, 'f': 0.6984126934744269}

Результаты для пары 3:
ROUGE-1: {'r': 0.4838709677419355, 'p': 0.5, 'f': 0.49180327368986837}
ROUGE-2: {'r': 0.22857142857142856, 'p': 0.26666666666666666, 'f': 0.24615384118343206}
ROUGE-L: {'r': 0.45161290322580644, 'p': 0.4666666666666667, 'f': 0.45901638844396675}

Результаты для пары 4:
ROUGE-1: {'r': 0.6206896551724138, 'p': 0.6666666666666666, 'f': 0.6428571378635205}
ROUGE-2: {'r': 0.41935483870967744, 'p': 0.4482758620689655, 'f': 0.43333332